In [134]:
import pandas as pd;

In [135]:
#connect
from sqlalchemy import create_engine

db_user = 'root'
db_password = 'hello'
db_host = '127.0.0.1'
db_name = 'sakila'
port='3306'
connectionStr=f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{port}/{db_name}"

engine = create_engine(connectionStr)



In [136]:
df=pd.read_sql("SELECT * FROM payment LIMIT 5",con=engine)
df.dropna()
df.drop_duplicates()
df

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
1,2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30
2,3,1,1,1185,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30
3,4,1,2,1422,0.99,2005-06-15 18:02:53,2006-02-15 22:12:30
4,5,1,2,1476,9.99,2005-06-15 21:08:46,2006-02-15 22:12:30


In [137]:
#extract
staff = pd.read_sql("SELECT * FROM staff", con=engine).dropna().drop_duplicates()
rent = pd.read_sql("SELECT rental_id, rental_date FROM rental", con=engine).dropna().drop_duplicates()
payment = pd.read_sql("SELECT staff_id, rental_id, payment_date, amount FROM payment", con=engine).dropna().drop_duplicates()


In [138]:
# transform date
payment['month'] = pd.to_datetime(payment['payment_date']).dt.to_period('M').dt.to_timestamp()
date_dim = pd.DataFrame({'date': payment['month'].drop_duplicates().sort_values().reset_index(drop=True)})
date_dim['year'] = date_dim['date'].dt.year
date_dim['month'] = date_dim['date'].dt.month
date_dim['day'] = date_dim['date'].dt.day
date_dim['date_id'] = date_dim.index + 1



In [139]:
#fact_monthly_payment
payment['month'] = pd.to_datetime(payment['payment_date']).dt.to_period('M').dt.to_timestamp()

fact_monthly_payment = payment.groupby(['staff_id', 'rental_id', 'month'])['amount'].sum().reset_index()


In [140]:
#join 
fact_monthly_payment = fact_monthly_payment.merge(date_dim[['date', 'date_id']], left_on='month', right_on='date', how='left')
fact_monthly_payment.drop(columns=['date'], inplace=True)


In [141]:
#load
staff.to_sql('dim_staff', con=engine, if_exists='replace', index=True)
rent.to_sql('dim_rent', con=engine, if_exists='replace', index=True)
date_dim.to_sql('dim_date', con=engine, if_exists='replace', index=True)
fact_monthly_payment[['staff_id', 'rental_id', 'date_id', 'amount']].to_sql('fact_monthly_payment', con=engine, if_exists='replace', index=True)

print("ETL is working well")

ETL is working well


In [142]:
fact_monthly_payment

,staff_id,rental_id,month,amount,date_id
0,1,1,2005-05-01,2.99,1
1,1,4,2005-05-01,4.99,1
2,1,5,2005-05-01,6.99,1
3,1,6,2005-05-01,0.99,1
4,1,9,2005-05-01,4.99,1
...,...,...,...,...,...
16039,2,16039,2005-08-01,2.99,4
16040,2,16040,2005-08-01,11.99,4
16041,2,16044,2005-08-01,0.99,4
16042,2,16048,2005-08-01,8.99,4
